In [5]:
import os
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Search
from folium.plugins import GroupedLayerControl

Map layer organization
1. All sites without moved ROIs
2. All sites with moved ROIs
3. FLOX sites without moved ROI
4. FLOX sites with moved ROIs

In [6]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"
cwd_Images_Results = cwd + "\\Results"

In [7]:
# Site
df = pd.read_excel(cwd_Images_Results + "\\Variogram - Fitting Results.xlsx")
df = df[~df['Site'].str.contains(' Moved')]
# No moved ROI
df_NoMoved = df[~df['Site'].str.contains(" New")].reset_index(drop = True)
# Only moved ROI
df_Moved = df[df['Site'].str.contains(" New")].reset_index(drop = True)
# FLOX no moved ROI
df_FLOX_NoMoved = df_NoMoved[df_NoMoved["Reference Network"].str.contains("FLOX")].reset_index(drop = True)
df_FLOX_NoMoved.head()

,Number,Site,Latitude,Longitude,Main Ecosystem,FLOX Activity,Reference Network,Variogram Fitting
0,4.0,AT-Mmg,47.3167,10.9703,Evergreen Forest,Yes,FLOX,Bad
1,6.0,BE-Bra,51.3076,4.5199,Evergreen Forest,Yes,FLOX-ICOS,Good
2,9.0,BE-WAL,50.5516,4.7461,Crop,No,FLOX,Bad
3,10.0,CANADA-ONT,42.7102,-80.3573,Evergreen Forest,Yes,FLOX,Good
4,12.0,CHINA-AGR 1,34.5152,115.5969,Crop,Not known,FLOX,Bad


In [8]:
# All sites without moved ROIs
df_All_NoMoved = df_NoMoved.copy()
gdf_All_NoMoved = gpd.GeoDataFrame(
    df_All_NoMoved, 
    geometry=gpd.points_from_xy(df_All_NoMoved.Longitude, df_All_NoMoved.Latitude), 
    crs = "EPSG:4326"
)
gdf_All_NoMoved.head(3)

,Number,Site,Latitude,Longitude,Main Ecosystem,FLOX Activity,Reference Network,Variogram Fitting,geometry
0,1.0,ATGE,52.466778,12.959778,NaN,NaN,HYPERNET,Good,POINT (12.95978 52.46678)
1,2.0,ATLAS-Mohammed V,33.406152,-5.103319,NaN,NaN,Other,Bad,POINT (-5.10332 33.40615)
2,4.0,AT-Mmg,47.316700,10.970300,Evergreen Forest,Yes,FLOX,Bad,POINT (10.9703 47.3167)


In [9]:
# All sites with moved ROIs
df_All_WithMoved = df.copy()
gdf_All_WithMoved = gpd.GeoDataFrame(
    df_All_WithMoved[['Site','Latitude','Longitude']], 
    geometry=gpd.points_from_xy(df_All_WithMoved.Longitude, df_All_WithMoved.Latitude), 
    crs = "EPSG:4326"
)
gdf_All_WithMoved.head(3)

,Site,Latitude,Longitude,geometry
0,ATGE,52.466778,12.959778,POINT (12.95978 52.46678)
1,ATLAS-Mohammed V,33.406152,-5.103319,POINT (-5.10332 33.40615)
3,ATLAS-Mohammed V New,33.404814,-5.101614,POINT (-5.10161 33.40481)


In [10]:
# FLOX sites without moved ROIs

gdf_FLOX_NoMoved = gpd.GeoDataFrame(
    df_FLOX_NoMoved, 
    geometry=gpd.points_from_xy(df_FLOX_NoMoved.Longitude, df_FLOX_NoMoved.Latitude), 
    crs = "EPSG:4326"
)
gdf_FLOX_NoMoved.head(3)

,Number,Site,Latitude,Longitude,Main Ecosystem,FLOX Activity,Reference Network,Variogram Fitting,geometry
0,4.0,AT-Mmg,47.3167,10.9703,Evergreen Forest,Yes,FLOX,Bad,POINT (10.9703 47.3167)
1,6.0,BE-Bra,51.3076,4.5199,Evergreen Forest,Yes,FLOX-ICOS,Good,POINT (4.5199 51.3076)
2,9.0,BE-WAL,50.5516,4.7461,Crop,No,FLOX,Bad,POINT (4.7461 50.5516)


In [11]:
# FLOX sites with moved ROIs
df_FLOX_WithMoved = df_All_WithMoved[df_All_WithMoved["Reference Network"].str.contains("FLOX")]
gdf_FLOX_WithMoved = gpd.GeoDataFrame(
    df_FLOX_WithMoved[['Site','Latitude','Longitude']], 
    geometry=gpd.points_from_xy(df_FLOX_WithMoved.Longitude, df_FLOX_WithMoved.Latitude), 
    crs = "EPSG:4326"
)
gdf_FLOX_WithMoved.head(3)

,Site,Latitude,Longitude,geometry
4,AT-Mmg,47.3167,10.9703,POINT (10.9703 47.3167)
6,BE-Bra,51.3076,4.5199,POINT (4.5199 51.3076)
9,BE-WAL,50.5516,4.7461,POINT (4.7461 50.5516)


In [12]:
# Other sites without moved ROIs
df_Other_NoMoved = df_All_NoMoved[~df_All_NoMoved["Reference Network"].str.contains("FLOX")]
gdf_Other_NoMoved = gpd.GeoDataFrame(
    df_Other_NoMoved, 
    geometry=gpd.points_from_xy(df_Other_NoMoved.Longitude, df_Other_NoMoved.Latitude), 
    crs = "EPSG:4326"
)
gdf_Other_NoMoved.head(3)

,Number,Site,Latitude,Longitude,Main Ecosystem,FLOX Activity,Reference Network,Variogram Fitting,geometry
0,1.0,ATGE,52.466778,12.959778,NaN,NaN,HYPERNET,Good,POINT (12.95978 52.46678)
1,2.0,ATLAS-Mohammed V,33.406152,-5.103319,NaN,NaN,Other,Bad,POINT (-5.10332 33.40615)
3,5.0,BASP,39.049139,-2.075917,NaN,NaN,HYPERNET,Good,POINT (-2.07592 39.04914)


# Prepare Shapefiles

In [13]:
def combine_Shp(df_Site, name_Shp, add_Shp = cwd_Images_Processed):
    for i in range(df_Site.shape[0]):
        temp_Name = df_Site.loc[i,"Site"]
        temp_gdf = gpd.read_file(add_Shp + "\\" + temp_Name + "\\" + name_Shp + ".shp").to_crs(epsg = 4326)
        if i == 0:
            temp_gdf_Final = temp_gdf.copy()
        else:
            temp_gdf_Final = pd.concat([temp_gdf_Final,temp_gdf])
    temp_gdf_Final.drop(columns = ['0'], inplace = True)
    temp_gdf_Final['Site'] = list(df_Site['Site'])
    temp_gdf_Final = temp_gdf_Final[['Site','geometry']]
    return temp_gdf_Final

In [14]:
gdf_30_NoMoved = combine_Shp(df_NoMoved, "30m")
gdf_100_NoMoved = combine_Shp(df_NoMoved, "100m")
gdf_300_NoMoved = combine_Shp(df_NoMoved, "300m")
gdf_600_NoMoved = combine_Shp(df_NoMoved, "600m")
gdf_900_NoMoved = combine_Shp(df_NoMoved, "900m")
gdf_30_Moved = combine_Shp(df_Moved, "30m")
gdf_100_Moved = combine_Shp(df_Moved, "100m")
gdf_300_Moved = combine_Shp(df_Moved, "300m")
gdf_600_Moved = combine_Shp(df_Moved, "600m")
gdf_900_Moved = combine_Shp(df_Moved, "900m")

# Create GIS Map

In [15]:
def getMarker(eco):
    match eco:
        case 'Evergreen Forest':
            return "EF"
        case 'Deciduous Forest':
            return "DF"
        case 'Wet':
            return 'W'
        case "Tundra":
            return "T"
        case "Grasslands":
            return "G"
        case "Savannah":
            return "S"
        case "Crop":
            return "C"

In [16]:
def testColor(result):
    if result == 'Yes':
        return 'green'
    else:
        return 'red'

In [17]:
m = folium.Map(location=[47.316500, 10.970300], zoom_start=4, tiles=None)

tile = folium.TileLayer(
    "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr='Esri', name="Esri Satellite",overlay = False
).add_to(m)

# Build folium geojson (site point)
geo_All_NoMoved = folium.GeoJson(
    gdf_All_NoMoved,
    name="All Sites",
    marker=folium.Marker(icon=folium.Icon(icon='star')),
    style_function = lambda x : {'markerColor':'green' if "FLOX" in x['properties']["Reference Network"] else 'blue'},
    tooltip=folium.GeoJsonTooltip(
        fields=["Site", "Latitude", "Longitude", "Reference Network"], localize=True
    ),
).add_to(m)

# Build folium geojson (site point)
geo_FLOX_NoMoved = folium.GeoJson(
    gdf_FLOX_NoMoved,
    name="FLOX Sites",
    marker=folium.Marker(icon=folium.Icon(icon='star')),
    style_function = lambda x : {'markerColor':'green',},
    tooltip=folium.GeoJsonTooltip(
        fields=["Site", "Latitude", "Longitude", "Main Ecosystem", "Reference Network"], localize=True
    ),
).add_to(m)

# Build folium geojson - Other Sites
geo_Other_NoMoved = folium.GeoJson(
    gdf_Other_NoMoved,
    name="Other Sites",
    marker=folium.Marker(icon=folium.Icon(icon='star')),
    style_function = lambda x : {'markerColor':'blue',},
    tooltip=folium.GeoJsonTooltip(
        fields=["Site", "Latitude", "Longitude", "Reference Network"], localize=True
    ),
).add_to(m)

# Build folium geojson (900m polygon)
geo_900m_NoMoved = folium.GeoJson(
    gdf_900_NoMoved,
    name="900m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Yellow'},
    popup = folium.Popup("900m x 900m!")
).add_to(m)

# Build folium geojson (300m polygon)
geo_600m_NoMoved = folium.GeoJson(
    gdf_600_NoMoved,
    name="600m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Green'},
    popup = folium.Popup("600m x 600m!")
).add_to(m)


# Build folium geojson (300m polygon)
geo_300m_NoMoved = folium.GeoJson(
    gdf_300_NoMoved,
    name="300m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Orange'},
    popup = folium.Popup("300m x 300m!")
).add_to(m)


# Build folium geojson (100m polygon)
geo_100m_NoMoved = folium.GeoJson(
    gdf_100_NoMoved,
    name="100m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Cyan'},
    popup = folium.Popup("100m x 100m!")
).add_to(m)

# Build folium geojson (30m polygon)
geo_30m_NoMoved = folium.GeoJson(
    gdf_30_NoMoved,
    name="30m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'White'},
    popup = folium.Popup("30m x 30m!")
).add_to(m)

# Moved ROIs
# Build folium geojson (900m polygon)
geo_900m_Moved = folium.GeoJson(
    gdf_900_Moved,
    name="900m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Yellow'},
    popup = folium.Popup("900m x 900m!"),
    show = False
).add_to(m)

# Build folium geojson (300m polygon)
geo_600m_Moved = folium.GeoJson(
    gdf_600_Moved,
    name="600m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Green'},
    popup = folium.Popup("600m x 600m!"),
    show = False
).add_to(m)


# Build folium geojson (300m polygon)
geo_300m_Moved = folium.GeoJson(
    gdf_300_Moved,
    name="300m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Orange'},
    popup = folium.Popup("300m x 300m!"),
    show = False
).add_to(m)


# Build folium geojson (100m polygon)
geo_100m_Moved = folium.GeoJson(
    gdf_100_Moved,
    name="100m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Cyan'},
    popup = folium.Popup("100m x 100m!"),
    show = False
).add_to(m)

# Build folium geojson (30m polygon)
geo_30m_Moved = folium.GeoJson(
    gdf_30_Moved,
    name="30m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'White'},
    popup = folium.Popup("30m x 30m!"),
    show = False
).add_to(m)

# Search
statesearch = Search(
    layer=geo_All_NoMoved,
    geom_type="Point",
    placeholder="Search for a site",
    collapsed=False,
    search_label="Site",
    search_zoom = 15,
    weight=3,
).add_to(m)

fg = folium.FeatureGroup(name="FLOX Homo Test", control=False).add_to(m)
# Build folium geojson (site point)
for i in range(df_FLOX_NoMoved.shape[0]):
    # print(i)
    temp_Name = df_FLOX_NoMoved['Site'][i]
    temp_Eco = df_FLOX_NoMoved['Main Ecosystem'][i]
    temp_Result = 'Yes' if df_FLOX_NoMoved['Homo Test 600'][i] == 'Yes' and df_FLOX_NoMoved['Homo Test 600'][i] == 'Yes' else 'No'
    temp_Lat = df_FLOX_NoMoved['Latitude'][i]
    temp_Lon = df_FLOX_NoMoved['Longitude'][i]
    temp_Network = df_FLOX_NoMoved['Reference Network'][i]
    # Set custom icon
    icon1 = folium.plugins.BeautifyIcon(
        # icon="circle", 
        border_color=testColor(temp_Result), 
        border_width = 0.5,
        text_color=testColor(temp_Result), 
        icon_shape="circle", 
        number = getMarker(temp_Eco),
        icon_size = [25,25],
        innerIconAnchor = [0,0],
        innerIconStyle = "font-size: 18px"
)
    folium.Marker(
        location = [temp_Lat,temp_Lon],
        popup = temp_Name,
        tooltip=folium.Tooltip(
            text=[temp_Name, temp_Lat, temp_Lon, temp_Eco, temp_Network]
        ),
        icon = icon1
        # icon = folium.Icon(icon = getMarker(temp_Eco), prefix= 'fa', color = testColor(temp_Result))
    ).add_to(fg)

folium.LayerControl().add_to(m)

GroupedLayerControl(
    groups={
        'Network': [geo_All_NoMoved, geo_FLOX_NoMoved, geo_Other_NoMoved, fg]
        },
    collapsed=False,
).add_to(m)

GroupedLayerControl(
    groups={
        'ROIs': [geo_30m_NoMoved, geo_100m_NoMoved, geo_300m_NoMoved, geo_600m_NoMoved, geo_900m_NoMoved]
        },
    exclusive_groups = False, 
    collapsed=False,
).add_to(m)

GroupedLayerControl(
    groups={
        'Moved ROIs': [geo_30m_Moved, geo_100m_Moved, geo_300m_Moved, geo_600m_Moved, geo_900m_Moved]
        },
    exclusive_groups = False, 
    collapsed=False,
).add_to(m)

m

KeyError: 'Homo Test 600'

In [14]:
m.save(cwd + "\\Site Map.html")

In [ ]:
# Quick export!!!
import io
from PIL import Image
for i in range(df.shape[0]):
    site = df.loc[i,'Site']
    lat = df.loc[i,'Latitude']
    lon = df.loc[i, 'Longitude']
    m = folium.Map(location=[lat, lon], zoom_start=15, tiles="Esri.WorldImagery")
    folium.TileLayer('MapQuest Open Aerial').add_to(m)

    # Build folium geojson (site point)
    geo_site = folium.GeoJson(
        gdf,
        name="Sites",
        # style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=["Site", "Latitude", "Longitude"], localize=True
        ),
    ).add_to(m)

    # Build folium geojson (1200m polygon)
    geo_1200m = folium.GeoJson(
        gdf_1200,
        name="1200m",
        style_function= lambda x: {'fillColor': '#00000000', 'color': 'Red'},
        popup = folium.Popup("1200m x 1200m!")
    ).add_to(m)

    # Build folium geojson (900m polygon)
    geo_900m = folium.GeoJson(
        gdf_900,
        name="900m",
        style_function= lambda x: {'fillColor': '#00000000', 'color': 'Yellow'},
        popup = folium.Popup("900m x 900m!")
    ).add_to(m)

    # Build folium geojson (300m polygon)
    geo_600m = folium.GeoJson(
        gdf_600,
        name="600m Moved",
        style_function= lambda x: {'fillColor': '#00000000', 'color': 'Green'},
        popup = folium.Popup("600m x 600m!")
    ).add_to(m)


    # Build folium geojson (300m polygon)
    geo_300m = folium.GeoJson(
        gdf_300,
        name="300m",
        style_function= lambda x: {'fillColor': '#00000000', 'color': 'Orange'},
        popup = folium.Popup("300m x 300m!")
    ).add_to(m)


    # Build folium geojson (100m polygon)
    geo_100m = folium.GeoJson(
        gdf_100,
        name="100m",
        style_function= lambda x: {'fillColor': '#00000000', 'color': 'Cyan'},
        popup = folium.Popup("100m x 100m!")
    ).add_to(m)

    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    img.save(cwd_Images_Processed + "\\" + site + "\\map.png")
    print(f"{site} Generated!")

ATGE Generated!
ATLAS-Mohammed V Generated!
AU-MIEMING Generated!
BASP Generated!
BE-BRASCHAAT Generated!
BE-Maa Generated!
BE-Vie Generated!
BE-WAL Generated!
CANADA-ONT Generated!
CD-Ygb Generated!
CHINA-AGR 1 Generated!
CHINA-AGR 2 Generated!
CHINA-FOR 1 Generated!
CZ-Lnz Generated!
Dahara (Senegal) Generated!
DEGE Generated!
DE-GEBESE Generated!
DE-Hai Generated!
DE-JUL1 Generated!
DE-KKA Generated!
DE-LEINIEFELDE Generated!
DE-LEIPZIG Generated!
DE-LEIPZIG 2 Generated!
DE-SEL Generated!
DE-Tha Generated!
DK-Sor Generated!
FI Hyt FOREST Generated!
FI SOD FOREST Generated!
FI SOD GRASS Generated!
FI-Ken Generated!
FINLAND SWAMP Generated!
FI-Var Generated!
FR-Bil Generated!
FR-FBn Generated!
FR-Hes Generated!
FR-OHP Generated!
FR-Pue Generated!
GF-Guy Generated!
GHNA Generated!
IFAR Generated!
INDONESIA Generated!
IN-JOD Generated!
IS-YATIR Generated!
IT-BFt Generated!
IT-BOZEN Generated!
IT-Cp2 Generated!
IT-Mbo Generated!
IT-SR2 Generated!
IT-TOR Generated!
JAES Generated!
KEN-1 G

In [19]:
import folium.features


m = folium.Map(location=[47.316500, 10.970300], zoom_start=4, tiles=None)

tile = folium.TileLayer(
    "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr='Esri', name="Esri Satellite",overlay = False
).add_to(m)



# Build folium geojson (site point)
fgall = folium.FeatureGroup(name="FLOX Homo Test", control=False).add_to(m)
# Build folium geojson (site point)
for i in range(df_FLOX_NoMoved.shape[0]):
    # print(i)
    temp_Name = df_FLOX_NoMoved['Site'][i]
    temp_Eco = df_FLOX_NoMoved['Main Ecosystem'][i]
    # temp_Result = 'Yes' if df_FLOX_NoMoved['Homo Test 600'][i] == 'Yes' and df_FLOX_NoMoved['Homo Test 600'][i] == 'Yes' else 'No'
    temp_Result = 'Yes'
    temp_Lat = df_FLOX_NoMoved['Latitude'][i]
    temp_Lon = df_FLOX_NoMoved['Longitude'][i]
    temp_Network = df_FLOX_NoMoved['Reference Network'][i]
    # Set custom icon
    smaller_Icon = folium.plugins.BeautifyIcon(
            # icon="circle", 
            border_color='Green', 
            border_width = 0.5,
            icon_shape="circle", 
            icon_size = [18,18],
            innerIconAnchor = [0,0]
    )
    folium.Marker(
        location = [temp_Lat,temp_Lon],
        popup = temp_Name,
        tooltip=folium.Tooltip(
            text=[temp_Name, temp_Lat, temp_Lon, temp_Eco, temp_Network]
        ),
        icon = smaller_Icon
        # icon = folium.Icon(icon = getMarker(temp_Eco), prefix= 'fa', color = testColor(temp_Result))
    ).add_to(fgall)

m

: 